In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import sample
import random
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls /content/gdrive/MyDrive/BibleModels/


char10eEpisode.pt      gpt2_medium_test_1.pt   word10eEpisode.pt
char10eRow.pt	       gpt2_medium_test_4.pt
gpt2_medium_test_0.pt  word10eEpisodelr001.pt


In [ ]:
!git clone https://github.com/HadasKedem/BibleGenerator.git
data = pd.read_csv('BibleGenerator/t_bbe.csv')


del data['id']
del data['c']
del data['v']
del data['b']
data.head()



In [ ]:
print(data)

In [ ]:
EOS_TOKEN = 0
MAX_SEQ_LEN = 100

class Vocab:
    def __init__(self):
        self.char2id = {}
        self.id2char = {}
        self.n_chars = 1
        
    def index_sentence(self, sentence):
      indexes = [self.index_char(c) for c in sentence]
      indexes.append(EOS_TOKEN)
      return indexes
    
    def index_char(self, c):
        if c not in self.char2id:
            self.char2id[c] = self.n_chars
            self.id2char[self.n_chars] = c
            self.n_chars += 1
        return self.char2id[c]
            
            
def prepare_data(data):
    vocab = Vocab()
    data_sequences = []
    for i in data.index:
        if len(data['t'][i]) <= MAX_SEQ_LEN:
            indexes = vocab.index_sentence(data['t'][i])
            data_sequences.append(indexes)
    print(data_sequences)
    return data_sequences, vocab



In [ ]:
data_sequences, vocab = prepare_data(data)

In [ ]:
class TextGen(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(TextGen, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, char_input, hidden):
        embedded = self.embedding(char_input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = self.out(output.view(1, -1))
        return output, hidden
    
    def init_hidden(self):
        return (torch.zeros(self.n_layers, 1, self.hidden_size).cuda(),
                torch.zeros(self.n_layers, 1, self.hidden_size).cuda())

In [ ]:
hidden_size = 800
n_layers = 2

# Initialize the model
model = TextGen(vocab.n_chars, hidden_size, vocab.n_chars, n_layers).cuda()

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (ADAM is a fancy version of SGD)
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:
def generate(model, vocab, start_string, temperature, max_len):
    '''
    This function gets a trained model and vocab and generates a random string
    using the model, seeded with the start_string string.
    The temparature value is used to generate a more diverse output (high value),
    then a conservative one (low value).
    '''
    str = start_string
    sequence = vocab.index_sentence(start_string)
        
    # remove the EOS, we don't need it for generation
    sequence = sequence[:-1]
    
    hidden = model.init_hidden()
    sequence_ten = torch.LongTensor(sequence).cuda()
    print("1")
    for i in range(len(sequence_ten) - 1):
      _, hidden = model(sequence_ten[i], hidden)
      
    output, hidden = model(sequence_ten[-1], hidden) 
    out_dist = output.view(-1).div(temperature).exp()
    print("1")
    new_c = vocab.id2char[torch.multinomial(out_dist, 1)[0].item()]
    str += new_c
    for i in range(max_len):
        new_c_var = torch.LongTensor([vocab.index_char(new_c)]).cuda()
        output, hidden = model(new_c_var, hidden)
        out_dist = output.view(-1).div(temperature).exp()
        char_id = torch.multinomial(out_dist, 1)[0].item()
        if char_id == EOS_TOKEN:
            return str
        new_c = vocab.id2char[char_id]
        str += new_c
    return str

Generating texts before training the model:

In [ ]:
for i in range(1):
    print(generate(model, vocab, 'h', 1, 200))

# The Training

In [ ]:
import time

start = time.time()

n_epochs = 5
print_every = 100
loss = 0
for e in range(1, n_epochs + 1):
    data_sequences_shuff = sample(data_sequences, len(data_sequences))
    
    for counter, sequence in enumerate(data_sequences_shuff):
      optimizer.zero_grad()
    
      seq_len = len(sequence)
      sequence_tensor = torch.LongTensor(sequence).cuda()
      
      sentence_loss = 0
      hidden = model.init_hidden()
      
      for i in range(seq_len - 1):
        output, hidden = model(sequence_tensor[i], hidden)
        sentence_loss += criterion(output.view(-1).unsqueeze(0), sequence_tensor[i + 1].unsqueeze(0))
    
      sentence_loss.backward()
      
      optimizer.step()
      
      loss += (sentence_loss.item() / seq_len)
      
      if counter % print_every == 0:
          loss = loss / print_every
          print('Epoch %d, %d/%d, Current Loss = %.4f' % (e, counter, len(data_sequences_shuff), loss))
          end = time.time()
          print(end - start)
          loss = 0
    
end = time.time()
print(end - start)

In [ ]:
model_save_name = 'word10eEpisodelr001.pt'
path = F"/content/gdrive/MyDrive/BibleModels/{model_save_name}" 
torch.save(model.state_dict(), path)

Generating text after 40 epochs of training

In [ ]:
for i in range(5):
    print(generate(model, vocab, 'Let', 1, 5000))

# Diving by episodes and generating episodes
opposite to dividing the data by rows

In [4]:
!git clone https://github.com/HadasKedem/BibleGenerator.git
data = pd.read_csv('BibleGenerator/t_bbe.csv')


del data['id']
del data['v']
del data['b']
data.head()


fatal: destination path 'BibleGenerator' already exists and is not an empty directory.


,c,t
0,1,At the first God made the heaven and the earth.
1,1,And the earth was waste and without form; and ...
2,1,"And God said, Let there be light: and there wa..."
3,1,"And God, looking on the light, saw that it was..."
4,1,"Naming the light, Day, and the dark, Night. An..."


In [5]:
def prepare_data_by_episodes(data):
    vocab = Vocab()
    data_sequences = []
    temp = ''
    c = 1
    for i in data.index:
        if len(data['t'][i]) <= MAX_SEQ_LEN:
            temp = temp + data['t'][i]
            if c != data['c'][i]:
                indexes = vocab.index_sentence(temp)
                data_sequences.append(indexes)
                temp = ''
                c = data['c'][i]
    return data_sequences, vocab

In [ ]:
data_sequences, vocab = prepare_data_by_episodes(data)
len(data_sequences)

In [ ]:


model_save_name = 'char10eEpisode.pt'
path = F"/content/gdrive/MyDrive/BibleModels/{model_save_name}" 
model.load_state_dict(torch.load(path))

In [ ]:
for i in range(5):
    print(generate(model, vocab, 'Let', 1, 5000))

# Word vocaulary

In [7]:
EOS_TOKEN = 0
MAX_SEQ_LEN = 1000

class Vocab_by_word:
    def __init__(self):
        self.word2id = {}
        self.id2word = {}
        self.n_words = 1
        
    def index_sentence(self, sentence):
      indexes = [self.index_char(word) for word in sentence.split()]
      indexes.append(EOS_TOKEN)
      return indexes
    
    def index_char(self, word):
        if word not in self.word2id:
            self.word2id[word] = self.n_words
            self.id2word[self.n_words] = word
            self.n_words += 1
        return self.word2id[word]

In [8]:
def prepare_data_by_episodes_word_vocab(data):
    vocab = Vocab_by_word()
    data_sequences = []
    temp = ''
    c = 1
    for i in data.index:
        if len(data['t'][i]) <= MAX_SEQ_LEN:
            temp = temp + data['t'][i]
            if c != data['c'][i]:
                indexes = vocab.index_sentence(temp)
                data_sequences.append(indexes)
                temp = ''
                c = data['c'][i]
    return data_sequences, vocab

In [26]:
data_sequences, vocab = prepare_data_by_episodes_word_vocab(data)
len(data_sequences)

1183

In [ ]:
hidden_size = 1000
n_layers = 2

# Initialize the model
model = TextGen(vocab.n_words, hidden_size, vocab.n_words, n_layers).cuda()

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (ADAM is a fancy version of SGD)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model_save_name = 'word10eEpisodelr001.pt'
path = F"/content/gdrive/MyDrive/BibleModels/{model_save_name}" 
model.load_state_dict(torch.load(path))

In [ ]:
def generate_episodes(model, vocab, start_string, temperature, max_len):
    '''
    This function gets a trained model and vocab and generates a random string
    using the model, seeded with the start_string string.
    The temparature value is used to generate a more diverse output (high value),
    then a conservative one (low value).
    '''
    str = start_string
    sequence = vocab.index_sentence(start_string)
        
    # remove the EOS, we don't need it for generation
    sequence = sequence[:-1]
    
    hidden = model.init_hidden()
    sequence_ten = torch.LongTensor(sequence).cuda()
    print("1")
    for i in range(len(sequence_ten) - 1):
      _, hidden = model(sequence_ten[i], hidden)
      
    output, hidden = model(sequence_ten[-1], hidden) 
    out_dist = output.view(-1).div(temperature).exp()
    print("1")
    new_c = vocab.id2word[torch.multinomial(out_dist, 1)[0].item()]
    str += new_c + " "
    for i in range(max_len):
        new_c_var = torch.LongTensor([vocab.index_char(new_c)]).cuda()
        output, hidden = model(new_c_var, hidden)
        out_dist = output.view(-1).div(temperature).exp()
        char_id = torch.multinomial(out_dist, 1)[0].item()
        if char_id == EOS_TOKEN:
            return str
        new_c = vocab.id2word[char_id] 
        str += new_c + " "
    return str


In [ ]:
for i in range(5):
    print(generate_episodes(model, vocab, 'Let', 1, 5000))

# Using GPT2

In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 10.7MB/s 
     |████████████████████████████████| 3.2MB 38.1MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f23938d6b18180847bcb13bc2bab11371547679bde238699675d79dc201ca272
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [11]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

class BibleDataset(Dataset):
    def __init__(self, dataset_path = '/content/gdrive/MyDrive/bibleData/'):
        super().__init__()

        short_path = os.path.join(dataset_path, 'bible.csv')

        self.bible_list = []
        self.eot_token = "<|endoftext|>"
        
        with open(short_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter="\t")
            
            x = 0
            for row in csv_reader:
                str = f"{row}{self.eot_token}"
                self.bible_list.append(str)
        
    def __len__(self):
        return len(self.bible_list)

    def __getitem__(self, item):
        return self.bible_list[item]
    
    def __replaceDS__(self, listDS):
        self.bible_list = listDS
        return self.bible_list

In [ ]:
def prepare_data_by_episodes(data):
    #vocab = Vocab()
    data_sequences = []
    temp = ''
    eot_token = "<|endoftext|>"
    c = 1
    for i in data.index:
        if len(data['t'][i]) <= MAX_SEQ_LEN:
            temp = temp + data['t'][i]
            if c != data['c'][i]:
                temp = temp + eot_token
                #indexes = vocab.index_sentence(temp)
                data_sequences.append(temp)
                temp = ''
                c = data['c'][i]
    return data_sequences #, vocab

data = prepare_data_by_episodes(data)
dataset = BibleDataset()
dataset = dataset.__replaceDS__(data)
bible_loader = DataLoader(dataset, batch_size=1, shuffle=True)
dataset

In [4]:
dataset = BibleDataset()
bible_loader = DataLoader(dataset, batch_size=1, shuffle=True)


In [5]:
BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 0.01
#WARMUP_STEPS = 500
MAX_SEQ_LEN = 700
#hidden_size = 800
vocab_size = 30978
n_layers = 2
from transformers import AdamW
import time

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'



# Training gpt2

In [9]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_tensor = None
models_folder = "/content/"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

start = time.time()
optimizer.zero_grad()
model.zero_grad()
for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} " )
    
    for idx,row in enumerate(bible_loader):
        
        #################### "Fit as many str sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        tensor = torch.tensor(tokenizer.encode(row[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if tensor.size()[1] > MAX_SEQ_LEN:
            continue
        
        #The first  str in the sequence
        if not torch.is_tensor(tmp_tensor):
            tmp_tensor = tensor
            continue
        else:
          #The next str does not fit in so we process the sequence and leave the last str 
            #as the start for next sequence 
            if tmp_tensor.size()[1] + tensor.size()[1] > MAX_SEQ_LEN:
                work_tensor = tmp_tensor
                tmp_tensor = tensor
            else:
                #Add the str to sequence, continue and try to add more
                tmp_tensor = torch.cat([tmp_tensor, tensor[:,1:]], dim=1)
                continue
        #Sequence ready, process it through the model 
        #print(work_tensor.shape)
        outputs = model(work_tensor)# ), labels=work_tensor)
        #print(outputs[1][0].size())
        print(outputs)
        #loss ,  hidden_states = outputs[:3]  
        loss , logits = outputs[:2]  
        print(loss)                      
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            end = time.time()/60
            print(end - start)
            batch_count = 0
            sum_loss = 0.0
    
    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_{epoch}.pt"))
    end = time.time()
    print(end - start)

EPOCH 0 
BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 0.7585,  0.0331, -1.1193,  ..., -0.3738, -0.1003, -0.3779],
         [-0.1642,  0.2114,  0.0543,  ..., -0.4708,  0.5184, -0.1552],
         [ 0.4666,  0.1848,  0.1875,  ..., -0.6161,  0.1079,  0.4181],
         ...,
         [ 1.8610,  0.0341,  0.2963,  ...,  0.4191, -0.2096,  0.1567],
         [ 1.7255,  0.1512,  0.1809,  ...,  0.2727, -0.3582,  0.2954],
         [ 0.2246, -0.0903,  0.1166,  ..., -0.0922, -0.0678, -0.0599]]],
       device='cuda:0', grad_fn=<ViewBackward>), past_key_values=((tensor([[[[ 4.4527e-01,  1.0741e-01,  1.9414e-01,  ..., -6.3121e-01,
            9.9626e-02,  1.2165e-01],
          [-1.7154e-01,  1.8818e-01,  6.7283e-02,  ...,  4.8637e-01,
           -2.1643e-01, -3.1199e-01],
          [ 4.3288e-01,  1.0258e-01,  1.6984e-01,  ..., -6.7979e-01,
            2.1602e-01, -2.7723e-01],
          ...,
          [-5.2201e-01, -3.3739e-02, -1.5669e-01,  ..., -2.4520e+00,
           -8.417

RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
model_save_name = 'gpt2_medium_test_6.pt'
path = F"/content/gdrive/MyDrive/BibleModels/{model_save_name}" 
torch.save(model.state_dict(), path)

In [18]:
!nvidia-smi



Sat Mar  6 08:42:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    27W /  70W |  15096MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
def choose_from_k_top(probs, k=5):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(k, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [ ]:
MODEL_EPOCH = 4
model = model.to(device)

models_folder = "/content/gdrive/MyDrive/BibleModels"

model_path = os.path.join(models_folder, f"gpt2_medium_test_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

output_file_path = f'generated_{MODEL_EPOCH}.rows'

model.eval()
if os.path.exists(output_file_path):
    os.remove(output_file_path)
    
num = 0
with torch.no_grad():
        for idx in range(5):
        
            finished = False

            cur_ids = torch.tensor(tokenizer.encode("And the world ")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    k = 20
                else:
                    k = 3
                next_token_id = choose_from_k_top(softmax_logits.to('cpu').numpy(), k=k) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|eot|>'):
                    finished = True
                    break

            
            if finished:
                
                num = num + 1
                
                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open(output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")
                    print(output_text)



#Output of gpt2 "And the world would" 4 epoches . LR = 0.01.  
And world would be judged by him, and his acts were upright.']<

And world would go after world, and all peoples would come to their end through need of food: and the Lord God gave them all their desire and their need, and they became the servants of his servants.']<

And world would be broken open, and the towns of the nations were made waste; and the people went after their haters and became food for themselves; and the land became a waste, and the trees of the wood were burned with fire: and they became dry land and without water.']<

And world would have no use for you: you were a cause of trouble and of trouble, and you were a cause of trouble and of trouble.']<

And world would go on and they went on; and all the men of the army went back to Jerusalem to take their rest there.']<